In [1]:
#general imports
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sklearn
import scipy.optimize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
import random
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

#fmclp imports
from fmclp import fmclp, cuae, synthetic_dataset,write_res,general_results_write, benefit, run_experiment

In [2]:
d = pd.read_csv('bar_pass_prediction.csv')

In [3]:
del d['ID']
del d['race1']
del d['race2']
del d['sex']
del d['bar']
del d['dnn_bar_pass_prediction']
del d['pass_bar']
del d['indxgrp2']
del d['gender']
del d['grad']
del d['Dropout']
del d['fulltime']
del d['lsat']
del d['zfygpa']
del d['ugpa']
del d['zgpa']
del d['other']
del d['asian']
del d['black']
del d['hisp']

In [4]:
def grouper_race(x):
    if x == 7:
        return 1
    else: 
        return 0
d['race'] = d['race'].apply(grouper_race)
def grouper_gpa(x):
    if x>3.4:
        return 2
    elif x <3.1:
        return 0
    else:
        return 1
d['gpa'] = d['gpa'].apply(grouper_gpa)
d = d.rename(columns = {'gpa':'target', 'race':'attr'})
d = pd.get_dummies(d,drop_first = True)
d = d.dropna(how = 'any')

In [5]:
d.columns

Index(['decile1b', 'decile3', 'decile1', 'attr', 'cluster', 'DOB_yr',
       'bar1_yr', 'bar2_yr', 'fam_inc', 'age', 'parttime', 'male',
       'bar_passed', 'tier', 'index6040', 'target', 'bar1_P', 'bar2_P',
       'indxgrp_b 400-460', 'indxgrp_c 460-520', 'indxgrp_d 520-580',
       'indxgrp_e 580-640', 'indxgrp_f 640-700', 'indxgrp_g 700+'],
      dtype='object')

In [33]:
y = d.drop(['target'], axis=1) 
x = d['target']
y_train,y_test,x_train,x_test = train_test_split(y,x)
lg = LGBMClassifier()
lg.fit(y_train, x_train)
lg = lg.predict(y_test)
accuracy_score(lg, x_test)

0.8339533972978265

In [6]:
%%time
experiment = run_experiment(dataset = d,
                   number_experiments = 100,
                   multiplier = 30,
                   number_iterations = 20,
                   interior_classifier = 'rf',
                   folder = 'gpa_race',
                   dataset_name='gpa_race',
                   initial_classifier = LGBMClassifier())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
CPU times: user 6d 9h 22min 44s, sys: 43min 53s, total: 6d 10h 6min 38s
Wall time: 6h 50min 33s


In [7]:
unfair_downgraded = np.array(
[0.14404762, 0.13260341 ,0.13965087, 0.13459119 ,0.1391201 , 0.14303483,
 0.15235792, 0.13620072 ,0.14002478, 0.14744352 ,0.13909774, 0.14371981,
 0.14105793, 0.14069767 ,0.14572193, 0.15640394 ,0.14002478, 0.12291933,
 0.13365735, 0.14465409 ,0.13365155, 0.16524909 ,0.13324709 ,0.14883149,
 0.13488372, 0.1322208  ,0.14025974, 0.13487242 ,0.13365155 ,0.14787879,
 0.15384615, 0.13452381 ,0.13882353, 0.12531328 ,0.14952153 ,0.12616822,
 0.15769712, 0.12408759 ,0.13704994, 0.14285714, 0.13431013 ,0.15770171,
 0.14585909, 0.14066194 ,0.13381995, 0.12742718, 0.14712919 ,0.13483146,
 0.15384615, 0.15235792 ,0.15662651, 0.14355231, 0.12515337 ,0.13658537,
 0.14459295, 0.15601966 ,0.15206813, 0.11904762, 0.13905683 ,0.13737624,
 0.14233577, 0.13365155 ,0.15375447, 0.14775414, 0.14634146 ,0.11571255,
 0.15834348, 0.15676359 ,0.14974619 ,0.1532567 , 0.14392991 ,0.15299878,
 0.13513514, 0.15294118 ,0.13793103, 0.13171355, 0.13744076 ,0.13942308,
 0.12718204, 0.13513514 ,0.14353499, 0.12515188, 0.12616822 ,0.13317479,
 0.12864078, 0.14200477 ,0.13447433, 0.15324675, 0.12911084 ,0.13032887,
 0.12899263, 0.1440678 , 0.14164649, 0.16525935, 0.14302885 ,0.14046392,
 0.14634146, 0.12799043, 0.15696203, 0.15834348])

In [8]:
fair_downgraded= np.array(
[0.11190476, 0.11435523 ,0.12718204, 0.10691824 ,0.10820452, 0.10199005,
 0.13784764, 0.12664277 ,0.14002478, 0.10701546 ,0.12155388, 0.12077295,
 0.12468514, 0.11511628 ,0.12032086, 0.11453202 ,0.12019827, 0.09859155,
 0.10935601, 0.11572327 ,0.1097852 , 0.13730255 ,0.10866753, 0.11685117,
 0.10930233, 0.10012837 ,0.11818182, 0.12150668 ,0.1097852 , 0.12121212,
 0.1001221 , 0.11428571 ,0.10823529, 0.10651629 ,0.12440191, 0.10163551,
 0.13516896, 0.09975669 ,0.1068525 , 0.11164466 ,0.11355311, 0.12591687,
 0.10506799, 0.12411348 ,0.10948905, 0.10558252 ,0.11842105, 0.10986267,
 0.1001221 , 0.11729141 ,0.13614458, 0.11800487 ,0.10674847, 0.12073171,
 0.11300122, 0.13390663 ,0.1216545 , 0.08452381 ,0.11245466, 0.12252475,
 0.12652068 ,0.11575179 ,0.11561383, 0.13475177 ,0.10801394, 0.0998782,
 0.11449452, 0.12010114 ,0.13959391, 0.1302682 , 0.12140175, 0.12117503,
 0.11547912, 0.13764706 ,0.11206897, 0.11381074, 0.11492891, 0.12379808,
 0.09850374, 0.11547912 ,0.11506524, 0.10206561 ,0.10163551, 0.09036861,
 0.10558252, 0.11455847 ,0.10880196, 0.11038961, 0.1181486 , 0.11693057,
 0.11793612, 0.12590799, 0.11501211, 0.14234017, 0.11778846, 0.12113402,
 0.12195122, 0.1076555,  0.12405063, 0.11449452])

In [10]:
unfair_downgraded.mean(),fair_downgraded.mean()

(0.1410011096, 0.11574491699999997)